## Imports & Co

In [ ]:
import pandas as pd
from sys import path
import requests as re
import folium
from folium import plugins
import datetime
import unicodedata
import unidecode

In [ ]:
data = pd.read_csv("person_2020_update.csv")

### Removing Accents Etc

In [ ]:
def remove_accents(s):
    return unidecode.unidecode(s)

In [ ]:
def more_accents(s):
    new_word = s.replace("à","a").replace("â","a").replace("æ","a").replace("á","a").replace("ä","a").replace("ã","a").replace("å","a").replace("ā","a")
    new_word = new_word.replace("é","e").replace("è","e").replace("ê","e").replace("ë","e").replace("ę","e").replace("ė","e").replace("ē","e")
    new_word = new_word.replace("î","i").replace("ï","i").replace("ì","i").replace("í","i").replace("į","i").replace("ī","i")
    new_word = new_word.replace("ô","o").replace("œ","o").replace("ö","o").replace("ò","o").replace("ó","o").replace("õ","o").replace("ø","o").replace("ō","o")                                                                                                                                            
    new_word = new_word.replace("ÿ","y")
    new_word = new_word.replace("û","u").replace("ù","u").replace("ü","u").replace("ú","u").replace("ū","u")
    return new_word                                                                                                                                        
                                                                                                                                                
                                                                                                                                                

In [ ]:
data["name2"] = data["name"].apply(remove_accents)

In [ ]:
data["name2"] = data["name2"].apply(more_accents)

### Adding Links

In [ ]:
data["link"] = data["name"]

In [ ]:
for i in range (0,len(data["slug"])):
    try: 
        link = f"https://en.wikipedia.org/wiki/{data['slug'][i]}"
        data["link"][i] = link    
    except KeyError:
        link = ""
        data["link"][i] = link

#### Colors

In [ ]:
board_games = {'CHESS PLAYER': "000000",'GO PLAYER':"2F4F4F", 'POKER PLAYER':"778899",'SNOOKER':"A9A9A9"}





In [ ]:
combat = ['BOXER':"4B0082", 'MARTIAL ARTS':"FFD700",'WRESTLER':"A52A2A",'BULLFIGHTER':"A52A2A",'FENCER':"A52A2A"]

In [ ]:
racket = ['TENNIS PLAYER':"6B8E23",'TABLE TENNIS PLAYER':"87CEFA",'BADMINTON PLAYER':"FFD700"]

In [ ]:
popular = {'SOCCER PLAYER':"#00FFFF", 'AMERICAN FOOTBALL PLAYER':"008B8B", \
           'BASKETBALL PLAYER':"EE82EE",'HANDBALL PLAYER':"F0E68C",'VOLLEYBALL PLAYER':"BDB76B",\
            'RUGBY PLAYER':"008B8B"}

In [ ]:
ice = {'SKIER':"87CEFA", 'HOCKEY PLAYER':"ADD8E6",'SWIMMER':"4169E1",'SKATER':"BDB76B"}

In [ ]:
other = ['COACH','REFEREE','GAMER']

In [ ]:
individual = {'ATHLETE':"#FF7F50", 'RACING DRIVER':"696969",\
              'CYCLIST':"FFFF00",'GYMNAST':"FFC0CB", 'GOLFER':"DEB887"]

In [ ]:
bat = {'BASEBALL PLAYER':"A52A2A",'CRICKETER:"F4A460"}

### Eliminating NAs in the List

In [ ]:
data[data["alive"] ==False]

In [ ]:
for i in range(0,len(data["name"])):
    try: 
        if "durant" in data["name"][i].lower(): 
            print(data[i])
    except KeyError:
        pass

In [ ]:
data["twitter"] = data['twitter'].fillna(0)
data["bplace_lat"] = data['bplace_lat'].fillna(0)
data["bplace_lon"] = data['bplace_lon'].fillna(0)

In [ ]:
na_list = []
count = 0
for i in data["bplace_lon"]:
    if i == 0:
        na_list.append(count)
        count += 1
    else:
        count += 1

In [ ]:
na_list_2 = []
count = 0
for i in data["bplace_lat"]:
    if i == 0:
        na_list_2.append(count)
        count += 1
    else:
        count += 1

In [ ]:
bad_people = []
for i in na_list: 
    if i in bad_people:
        pass
    else:
        bad_people.append(i)

In [ ]:
for i in na_list_2: 
    if i in bad_people:
        pass
    else:
        bad_people.append(i)

In [ ]:
data = data[data.bplace_lat != 0]
data = data[data.bplace_lon != 0]

## Mapping

In [ ]:
color = {'CHESS PLAYER': "000000",'GO PLAYER':"2F4F4F", 'POKER PLAYER':"778899",'SNOOKER':"A9A9A9",\
        'BOXER':"4B0082", 'MARTIAL ARTS':"FFD700",'WRESTLER':"A52A2A",'BULLFIGHTER':"A52A2A",'FENCER':"A52A2A",\
        'TENNIS PLAYER':"6B8E23",'TABLE TENNIS PLAYER':"87CEFA",'BADMINTON PLAYER':"FFD700",\
        'SOCCER PLAYER':"#00FFFF", 'AMERICAN FOOTBALL PLAYER':"008B8B", \
        'BASKETBALL PLAYER':"EE82EE",'HANDBALL PLAYER':"F0E68C",'VOLLEYBALL PLAYER':"BDB76B",\
        'RUGBY PLAYER':"008B8B",'SKIER':"87CEFA", 'HOCKEY PLAYER':"ADD8E6",'SWIMMER':"4169E1",\
        'SKATER':"BDB76B",'ATHLETE':"#FF7F50", 'RACING DRIVER':"696969", 'CYCLIST':"FFFF00",\
        'GYMNAST':"FFC0CB", 'GOLFER':"DEB887",'BASEBALL PLAYER':"A52A2A",'CRICKETER':"F4A460",\
        'COACH':"7B68EE",'REFEREE':"7B68EE",'GAMER':"FF69B4",}

In [ ]:
lat = data["bplace_lat"]
long = data["bplace_lon"]

In [ ]:
issues = []
year_map = folium.Map(location=(50.850346,4.351721),zoom_start=3)
for i in range(0,88936):
    if i % 10000 == 0:
        print(i)
    try: 
        coordinates = (float(lat[i]), float(long[i]))
        folium.CircleMarker(
            popup=folium.Popup(html=f"<h4 style='color:black';><a href={str(data['link'][i])}>{str(data['name2'][i])}</a></h4><body style='color:black';>Gender: {str(data['gender'][i])} Age: {str(data['age'][i])} Birthdate:{str(data['birthdate'][i])}  Occupation: {str(data['occupation'][i])} </body>",max_width="50%"),
            tooltip=f"{str(data['name2'][i])}, {data['occupation'][i]}",
            radius=3,
            location=coordinates,
            color=f'#{color[data["occupation"][i]]}',
            fill=True,
        ).add_to(year_map)
    except KeyError:
        issues.append(i)
year_map

In [ ]:
year_map

In [ ]:
year_map.save('MySportsMap4.html')

In [ ]:
ls